In [ ]:
import pandas as pd

# Ler o dataset
df = pd.read_csv('/content/Dados.csv.csv')

# Verificar os primeiros registros
print(df.head())

# Resumo estatístico
print(df.describe())

# Verificar tipos de dados
print(df.dtypes)


In [ ]:
# Verificar valores nulos no CustomerID
print(df['CustomerID'].isna().sum())

# Remover valores nulos
df = df.dropna(subset=['CustomerID'])


In [ ]:
# Verificar se existem preços menores ou iguais a 0
print(df[df['UnitPrice'] <= 0].shape)

# Remover preços menores ou iguais a 0
df = df[df['UnitPrice'] > 0]

# Verificar se existem quantidades menores ou iguais a 0
print(df[df['Quantity'] <= 0].shape)

# Remover quantidades menores ou iguais a 0
df = df[df['Quantity'] > 0]


In [ ]:
# Verificar duplicados
print(df.duplicated().sum())

# Remover duplicados
df = df.drop_duplicates()


In [ ]:
# Ajustar tipo de CustomerID para inteiro
df['CustomerID'] = df['CustomerID'].astype(int)

# Converter InvoiceDate para formato datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])


In [ ]:
# Remover outliers de quantidade e preço unitário
df = df[(df['Quantity'] <= 10000) & (df['UnitPrice'] <= 5000)]


In [ ]:
# Criar coluna de preço total
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']


In [ ]:
# Última data de compra
last_date = df['InvoiceDate'].max()
print(last_date)


In [ ]:
!pip install plotly


**Top 10 países com maior valor em vendas**

In [ ]:
import plotly.express as px

# Agrupar por país e calcular o valor total de vendas
vendas_pais = df.groupby('Country')['TotalPrice'].sum().sort_values(ascending=False).head(10).reset_index()

# Criar o gráfico interativo
fig = px.bar(vendas_pais, x='Country', y='TotalPrice',
             title='Top 10 Países com Maior Valor em Vendas',
             labels={'TotalPrice': 'Total de Vendas', 'Country': 'País'},
             template='plotly')

fig.show()


**Top 10 produtos mais vendidos**

In [ ]:
# Agrupar por produto (Description) e calcular a quantidade total vendida
produtos_vendidos = df.groupby('Description')['Quantity'].sum().sort_values(ascending=False).head(10).reset_index()

# Criar o gráfico interativo
fig = px.bar(produtos_vendidos, x='Description', y='Quantity',
             title='Top 10 Produtos Mais Vendidos',
             labels={'Quantity': 'Quantidade Vendida', 'Description': 'Produto'},
             template='plotly')

fig.update_layout(xaxis_tickangle=-45)
fig.show()


**Valor de venda total por mês**

In [ ]:
# Criar uma coluna com o ano e mês
df['YearMonth'] = df['InvoiceDate'].dt.to_period('M').astype(str)

# Agrupar por mês e somar o total de vendas
vendas_mes = df.groupby('YearMonth')['TotalPrice'].sum().reset_index()

# Criar o gráfico interativo
fig = px.line(vendas_mes, x='YearMonth', y='TotalPrice',
              title='Valor de Venda Total por Mês',
              labels={'TotalPrice': 'Total de Vendas', 'YearMonth': 'Mês'},
              template='plotly')

fig.update_layout(xaxis_tickangle=-45)
fig.show()


**Valor de venda total por mês e por país (Top 10 países)**

In [ ]:
# Filtrar para os top 10 países com maior valor de vendas
top_paises = df.groupby('Country')['TotalPrice'].sum().sort_values(ascending=False).head(10).index

# Filtrar o dataset para esses países
df_top_paises = df[df['Country'].isin(top_paises)]

# Agrupar por mês e país, e somar o total de vendas
vendas_mes_pais = df_top_paises.groupby(['YearMonth', 'Country'])['TotalPrice'].sum().reset_index()

# Criar o gráfico interativo
fig = px.line(vendas_mes_pais, x='YearMonth', y='TotalPrice', color='Country',
              title='Valor de Venda Total por Mês e por País (Top 10 Países)',
              labels={'TotalPrice': 'Total de Vendas', 'YearMonth': 'Mês'},
              template='plotly')

fig.update_layout(xaxis_tickangle=-45)
fig.show()


In [ ]:
# Agrupar por CustomerID e InvoiceNo para obter a data da compra e o preço total por pedido
df_pedidos = df.groupby(['CustomerID', 'InvoiceNo']).agg({
    'InvoiceDate': 'max',   # Data da última compra por fatura
    'TotalPrice': 'sum'     # Soma do preço total por fatura
}).reset_index()

df_pedidos.head()


In [ ]:
# Calcular a última data disponível no dataset
ultima_data = df['InvoiceDate'].max()

print(f"A última data no conjunto de dados é: {ultima_data}")


In [ ]:
# Agrupar por cliente para calcular o RFM
df_rfm = df_pedidos.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (ultima_data - x.max()).days,  # Recência
    'InvoiceNo': 'count',                                  # Frequência
    'TotalPrice': 'mean'                                   # Ticket Médio (Monetary)
}).reset_index()

# Renomear as colunas para R, F e M
df_rfm.columns = ['CustomerID', 'R', 'F', 'M']

df_rfm.head()
